In [15]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

In [79]:
n = 3
s = [(0,0),(0,0)]

def getMap(maps={}, needs='a', maxA=3, cash=()):
    if len(cash) == 0:
        n = maxA - 1
        s = [(0,0) for _ in range(maxA)]
    else:
        n = cash[0]
        s = cash[1]
        
    if n == 0:
        for i in range(5):
            for j in range(5):
                s[n] = (i,j)
                if needs == 'a':
                    maps[str(s)] = [0]*maxA
                else:
                     maps[str(s)] = 0
    else:
        for i in range(5):
            for j in range(5):
                s[n] = (i,j)
                getMap(maps, needs, maxA, (n-1,s.copy()))
    return maps

In [85]:
actions = getMap(maps={}, needs='a', maxA=3)
values = getMap(maps={}, needs='v', maxA=3)

In [86]:
len(actions)

15625

In [82]:
list(values.keys())[0]

'[(0, 0), (0, 0)]'

In [83]:
actions[str([(3, 4), (1, 2)])]

[0, 0]